In [ ]:
# import os
# os.environ['HF_ENDPOINT'] = 'https://hf-mirror.com'

In [ ]:
import subprocess
import os

result = subprocess.run('bash -c "source /etc/network_turbo && env | grep proxy"', shell=True, capture_output=True, text=True)
output = result.stdout
for line in output.splitlines():
    if '=' in line:
        var, value = line.split('=', 1)
        os.environ[var] = value

In [ ]:
#1安装微调库
# %%capture
import torch
major_version, minor_version = torch.cuda.get_device_capability()
# 由于Colab有torch 2.2.1，会破坏软件包，要单独安装
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install --no-deps xformers trl peft accelerate bitsandbytes
# if major_version >= 8:
#     # 新GPU，如Ampere、Hopper GPU（RTX 30xx、RTX 40xx、A100、H100、L40）。
#     !pip install --no-deps packaging ninja einops flash-attn xformers trl peft accelerate bitsandbytes
# else:
#     # 较旧的GPU（V100、Tesla T4、RTX 20xx）
#     !pip install --no-deps xformers trl peft accelerate bitsandbytes
# pass

In [ ]:
#2加载模型
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048
dtype = None
load_in_4bit = True
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/llama-3-8b-Instruct-bnb-4bit",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

In [ ]:
#3. 微调前测试
alpaca_prompt = """
{}
### Instruction:
{}
### Input:
{}
### Response 1:
{}
### Response 2:
{}
### Evaluation:
{}
### Reason:
{}
"""
FastLanguageModel.for_inference(model)
inputs = tokenizer(
[
    alpaca_prompt.format(
        "Below are two responses for a given task. The task is defined by the Instruction. Evaluate the responses and generate a reference answer for the task.", # prompt
        "Give three tips for staying healthy.", # instruction
        "", #input 
        "1. Eat a balanced and nutritious diet.\n2. Get regular exercise.\n3. Get enough sleep.", # response 1
        "1. Eat a balanced diet with plenty of fruits, vegetables, and whole grains.\n2. Get regular physical activity, such as walking, jogging, or swimming.\n3. Get enough sleep and practice healthy sleeping habits.", # response 2
        "", # evalutaion
        "", # reason
    )
], return_tensors = "pt").to("cuda")
from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer)
_ = model.generate(**inputs, streamer = text_streamer, max_new_tokens =128)


In [ ]:
from IPython.core.debugger import set_trace
#4. 准备微调数据集
EOS_TOKEN = tokenizer.eos_token # 必须添加 EOS_TOKEN
def formatting_prompts_func(examples):
    prompts      = examples["prompt"]
    instructions = examples["instruction"]
    inputs       = examples["input"]
    response_1s  = examples["response_1"]
    response_2s  = examples["response_2"]
    evaluations  = examples["evaluation"]
    reasons      = examples["reason"]
    texts = []
    for prompt,instruction, input, response_1,response_2,evaluation,reason in zip(prompts,instructions, inputs, response_1s,response_2s,evaluations,reasons):
        # 必须添加EOS_TOKEN，否则无限生成
        text = alpaca_prompt.format(prompt+"Please ensure the evaluation and reason are consistent upon repeated inquiries.", instruction, input, response_1,response_2,evaluation,reason) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts, }
pass

from datasets import load_dataset
# dataset = load_dataset("Lemon01/test_data", split = "train")
dataset = load_dataset('json', data_files='autodl-tmp/pandalm_after_preprocess_v2.json')
dataset = dataset.map(formatting_prompts_func, batched = True,)

In [ ]:
# print(dataset['train'][0])

In [10]:
#5. 设置训练参数
from trl import SFTTrainer
from transformers import TrainingArguments

# model = FastLanguageModel.get_peft_model(
#     model,
#     r = 16, #  建议 8, 16, 32, 64, 128
#     target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
#                       "gate_proj", "up_proj", "down_proj",],
#     lora_alpha = 32,
#     lora_dropout = 0,
#     bias = "none",
#     use_gradient_checkpointing = "unsloth", # 检查点，长上下文度
#     random_state = 3407,
#     use_rslora = False,
#     loftq_config = None,
# )

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset['train'],
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False, # 可以让短序列的训练速度提高5倍。
    args = TrainingArguments(
        per_device_train_batch_size = 64,
        gradient_accumulation_steps = 3,
        warmup_steps = 100,
        max_steps = 1500,  # 微调步数
        learning_rate = 5e-4, # 学习率
        fp16 = not torch.cuda.is_bf16_supported(),
        bf16 = torch.cuda.is_bf16_supported(),
        logging_steps = 50,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
    ),
)

max_steps is given, it will override any value given in num_train_epochs


In [ ]:
#6. 开始训练
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 274,081 | Num Epochs = 2
O^O/ \_/ \    Batch size per device = 64 | Gradient Accumulation steps = 3
\        /    Total batch size = 192 | Total steps = 1,500
 "-____-"     Number of trainable parameters = 41,943,040


Step,Training Loss


In [ ]:
FastLanguageModel.for_inference(model)
inputs = tokenizer(
[
    alpaca_prompt.format(
        "Below are two responses for a given task. The task is defined by the Instruction. Evaluate the responses and generate a reference answer for the task.", # prompt
        "Give three tips for staying healthy.", # instruction
        "", #input 
        "1. Eat a balanced and nutritious diet.\n2. Get regular exercise.\n3. Get enough sleep.", # response 1
        "1. Eat a balanced diet with plenty of fruits, vegetables, and whole grains.\n2. Get regular physical activity, such as walking, jogging, or swimming.\n3. Get enough sleep and practice healthy sleeping habits.", # response 2
        "", # evalutaion
        "", # reason
    )
], return_tensors = "pt").to("cuda")
from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer)
_ = model.generate(**inputs, streamer = text_streamer, max_new_tokens =128)

In [ ]:
#8保存LoRA模型
model.save_pretrained("autodl-tmp/lora_model") # Local saving
# model.push_to_hub("your_name/lora_model", token = "...") # 在线保存到hugging face，需要token

In [ ]:
# 保存tokenizer
tokenizer.save_pretrained("autodl-tmp/tokenizer")

In [ ]:
# model.from_pretrained(model,"autodl-tmp/lora_model")

In [ ]:
#9合并模型并量化成4位gguf保存
model.save_pretrained_gguf("autodl-tmp/model", tokenizer, quantization_method = "q4_k_m")
#model.save_pretrained_merged("outputs", tokenizer, save_method = "merged_16bit",) #合并模型，保存为16位hf
#model.push_to_hub_gguf("hf/model", tokenizer, quantization_method = "q4_k_m", token = "") #合并4位gguf，上传到hugging face(需要账号token)